In [1]:
import json
import os
from tqdm import tqdm
import numpy as np
import torch
# GPU 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
# Rule load = document = corpus
data = 'data/aihub_rules_prev.json'
with open(data, 'r', encoding="UTF-8") as j:
    aihub_rule = json.load(j)

normal_rule = aihub_rule['normal']
abnormal_rule = aihub_rule['abnormal']
combined_rules = normal_rule + abnormal_rule

In [3]:
# Caption load = discreption = query
root_path = '/data1/sliver/jwsuh/construction_dataset/aihub/llava/llava_image_result_with_obj'
files = sorted([i for i in os.listdir(root_path) if i.endswith('.json')])

files_captions = []
for idx, file in enumerate(tqdm(files)):
    with open(os.path.join(root_path, file), 'r', encoding="UTF-8") as j:
        caption = json.load(j)['outputs']
    files_captions.append(caption)
    

100%|██████████| 943/943 [00:00<00:00, 24455.45it/s]


In [4]:
def find_rank_of_answer_in_results(final_indices, answer_index):
    try:
        return final_indices.index(answer_index) + 1
    except ValueError:
        return -1

In [5]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer("all-MiniLM-L6-v2")  # 우선 임시로 사용할 모델

corpus_embeddings = embedder.encode(combined_rules, convert_to_tensor=True)
query_embedding = embedder.encode(files_captions, convert_to_tensor=True)

In [6]:
from tqdm import tqdm
from sentence_transformers.cross_encoder import CrossEncoder

model = CrossEncoder("cross-encoder/stsb-roberta-large", device=device, max_length=512)


In [7]:
save_right_index = {}
save_wrong_index = {}
biencoder_and_cross_encoder_final_top_k_indices = []

for final_top_k in [50]:
    correct = 0
    save_right_index[final_top_k] = {}
    save_wrong_index[final_top_k] = {}
    
    for idx, file in enumerate(tqdm(files)):
        with open(os.path.join(root_path, file), 'r', encoding="UTF-8") as j:
            caption = json.load(j)
        
        tokenized_query = query_embedding[idx]
        initial_top_k_scores = util.cos_sim(tokenized_query, corpus_embeddings)[0]
        top_results =  torch.topk(initial_top_k_scores, k=50)
        initial_top_k_docs = [combined_rules[i] for i in top_results[1]]
        
        sentence_combinations = [[caption['outputs'], doc] for doc in initial_top_k_docs]
        reranked_scores = model.predict(sentence_combinations)
        reranked_indices = np.argsort(reranked_scores)[::-1]
        final_top_k_indices = [top_results[1][i] for i in reranked_indices]
        
        biencoder_and_cross_encoder_final_top_k_indices.append(final_top_k_indices)
        
        answer = file.split('_')[2]
        if answer[0] == 'Y':
            answer_index = int(answer[2:]) - 1
        elif answer[0] == 'N':
            answer_index = int(answer[2:]) + 49
            
        rank = find_rank_of_answer_in_results(final_top_k_indices, answer_index)

        if rank <= final_top_k and rank != -1:
            correct += 1
            save_right_index[final_top_k][idx] = final_top_k_indices
        else:
            save_wrong_index[final_top_k][idx] = final_top_k_indices

    print(f"Top-{final_top_k} accuracy:", correct / len(files))

100%|██████████| 943/943 [12:07<00:00,  1.30it/s]

Top-50 accuracy: 0.9554612937433722


In [8]:
for final_top_k in [1, 5, 10, 15, 20, 25, 30]:
    correct = 0
    
    for idx, file in enumerate(tqdm(files)):
        with open(os.path.join(root_path, file), 'r', encoding="UTF-8") as j:
            caption = json.load(j)
        
        final_top_k_indices = biencoder_and_cross_encoder_final_top_k_indices[idx]

        answer = file.split('_')[2]
        if answer[0] == 'Y':
            answer_index = int(answer[2:]) - 1
        elif answer[0] == 'N':
            answer_index = int(answer[2:]) + 49
            
        rank = find_rank_of_answer_in_results(final_top_k_indices, answer_index)

        if rank <= final_top_k and rank != -1:
            correct += 1

    print(f"Top-{final_top_k} accuracy:", correct / len(files))

100%|██████████| 943/943 [00:00<00:00, 1880.18it/s]


Top-1 accuracy: 0.20784729586426298


100%|██████████| 943/943 [00:00<00:00, 1929.91it/s]


Top-5 accuracy: 0.4687168610816543


100%|██████████| 943/943 [00:00<00:00, 1822.78it/s]


...

Top-20 accuracy: 0.7242841993637328


100%|██████████| 943/943 [00:00<00:00, 1965.06it/s]


Top-25 accuracy: 0.777306468716861


100%|██████████| 943/943 [00:00<00:00, 1962.01it/s]

Top-30 accuracy: 0.8303287380699894
